In [1]:
import scanpy as sc
import pandas as pd
import os
from scipy.io import mmread
from scipy.sparse import coo_matrix
import scglue
import numpy as np
from matplotlib import rcParams

/data/gaojie/miniforge3/envs/GLUE/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.chdir('/home/gaojie/workspace/Mida_collab/')
rcParams["figure.figsize"] = (4, 4)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

### preparing RNA

In [3]:
### Raw counts matrix, cells, genes are exported from our Seurat scRNA-seq data
RNA_raw_matrix = mmread('Multi-omics_intergration/data/RNA_raw_counts.mtx')
cell_names = pd.read_csv('Multi-omics_intergration/data/RNA_raw_counts_cells.csv')
gene_names = pd.read_csv('Multi-omics_intergration/data/RNA_raw_counts_genes.csv')

In [4]:
RNA_raw_matrix.T

<28272x36601 sparse matrix of type '<class 'numpy.int64'>'
	with 101637381 stored elements in COOrdinate format>

In [5]:
###Constructing RNA adata from raw matrix, basic quality control
rna = sc.AnnData(X=RNA_raw_matrix.T.tocsr())
rna.obs_names = cell_names.iloc[:,1].tolist()
rna.var_names = gene_names.iloc[:,1].tolist()
rna.var_names_make_unique()
rna.obs_names_make_unique()
rna.var["mt"] = rna.var_names.str.startswith("MT-")
rna.var["ribo"] = rna.var_names.str.startswith(("RPS", "RPL"))
rna.var["hb"] = rna.var_names.str.contains("^HB[^(P)]")
sc.pp.calculate_qc_metrics(
    rna, qc_vars=["mt", "ribo", "hb"], inplace=True, log1p=True
)
sc.pp.filter_cells(rna, min_genes=100)
sc.pp.filter_genes(rna, min_cells=3)

In [6]:
rna

AnnData object with n_obs × n_vars = 28272 × 30705
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'n_genes'
    var: 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells'

In [7]:
rna.layers["counts"] = rna.X.copy() ###save raw counts

In [8]:
rna.obs['sample'] = [x.split('_')[0]+'_'+x.split('_')[1] for x in rna.obs_names] ###adding region info from barcodes
rna.obs.head()

,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,total_counts_mt,log1p_total_counts_mt,pct_counts_mt,total_counts_ribo,log1p_total_counts_ribo,pct_counts_ribo,total_counts_hb,log1p_total_counts_hb,pct_counts_hb,n_genes,sample
GW11_C_AAACCCAAGCCGATTT-1,5128,8.542666,17048,9.743847,16.570859,23.451431,32.426091,47.671281,219,5.393628,1.284608,902,6.805723,5.290943,5,1.791759,0.029329,5128,GW11_C
GW11_C_AAACCCAAGGTTGTTC-1,2923,7.980708,6199,8.732305,16.260687,24.084530,34.150669,51.234070,103,4.644391,1.661558,587,6.376727,9.469269,10,2.397895,0.161316,2923,GW11_C
GW11_C_AAACCCAAGTGTGTTC-1,2682,7.894691,4846,8.486115,14.403632,21.027652,30.148576,46.698308,19,2.995732,0.392076,452,6.115892,9.327280,10,2.397895,0.206356,2682,GW11_C
GW11_C_AAACCCAGTCCCGTGA-1,4053,8.307459,11046,9.309914,16.657614,24.144487,33.722615,49.755568,193,5.267858,1.747239,1033,6.941190,9.351802,4,1.609438,0.036212,4053,GW11_C
GW11_C_AAACCCAGTTTGAAAG-1,1188,7.080868,1739,7.461640,17.423807,26.049454,38.355377,60.437033,72,4.290459,4.140311,37,3.637586,2.127660,2,1.098612,0.115009,1188,GW11_C


In [9]:
rna.raw = rna

In [10]:
sc.pp.normalize_total(rna)
sc.pp.log1p(rna)
sc.pp.highly_variable_genes(adata=rna,n_top_genes=4000,batch_key='sample')

In [11]:
###adding all TFs as input because GLUE only takes highly variable genes into account
TF_list = pd.read_csv('Multi-omics_intergration/TF_names_v_1.01.txt',header=None)
rna.var['TF'] = [(i in TF_list[0].tolist()) for i in rna.var_names]
rna.var['TF'].sum()

1506

In [12]:
RGG_df = pd.read_csv('CellOracle/gradient_table2.vRG.tsv',sep='\t')
RGG = RGG_df[RGG_df['flag']=='Pass']['gene']
rna.var['RGG'] = [(i in RGG.tolist()) for i in rna.var_names]
rna.var.loc[RGG,'highly_variable'] = True

In [13]:
###Filtering TFs based on Gene Expression and Expression Percentage in cells
expression = pd.DataFrame((np.sum(rna.X > 0, axis=0))/(rna.shape[0])).T
expression.index = rna.var_names
rna.var['exp_ratio'] = expression[0]
for i in rna.var[rna.var['TF']==True].index:
    if (rna.var.loc[i,'exp_ratio']>=0.05):
        rna.var.loc[i,'highly_variable'] = True

In [14]:
sc.tl.pca(rna)
sc.external.pp.harmony_integrate(rna,key='sample')
sc.pp.neighbors(rna,use_rep='X_pca_harmony')
sc.tl.umap(rna)

2025-09-24 06:02:42,646 - harmonypy - INFO - Computing initial centroids with sklearn.KMeans...
2025-09-24 06:02:48,242 - harmonypy - INFO - sklearn.KMeans initialization complete.
2025-09-24 06:02:48,314 - harmonypy - INFO - Iteration 1 of 10
2025-09-24 06:02:51,260 - harmonypy - INFO - Iteration 2 of 10
2025-09-24 06:02:54,295 - harmonypy - INFO - Iteration 3 of 10
2025-09-24 06:02:57,018 - harmonypy - INFO - Iteration 4 of 10
2025-09-24 06:02:58,879 - harmonypy - INFO - Iteration 5 of 10
2025-09-24 06:03:01,540 - harmonypy - INFO - Iteration 6 of 10
2025-09-24 06:03:04,648 - harmonypy - INFO - Iteration 7 of 10
2025-09-24 06:03:06,297 - harmonypy - INFO - Converged after 7 iterations


In [15]:
celltypes = pd.read_csv('sample_data/RNA_raw/annotation.csv',header=0,index_col=0) ###adding RNA annotation
rna.obs['celltype'] = celltypes.loc[rna.obs_names,'celltype']
rna.obs['omic'] = 'RNA'

In [16]:
scglue.data.get_gene_annotation(
    rna, gtf="/home/gaojie/workspace/reference/refdata-gex-GRCh38-2020-A/genes/genes.gtf",
    gtf_by="gene_name"
)
rna.var.loc[:, ["chrom", "chromStart", "chromEnd"]].head()
rna = rna[:,rna.var['gene_id'].isna()==False] ###remove unannotated genes
rna

View of AnnData object with n_obs × n_vars = 28272 × 30696
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'n_genes', 'sample', 'celltype', 'omic'
    var: 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection', 'TF', 'RGG', 'exp_ratio', 'chrom', 'chromStart', 'chromEnd', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts', 'gene_id', 'gene_version', 'gene_type', 'hg

In [17]:
rna = rna[~rna.obs['celltype'].isin(['Others','?','Others_to_remove']),:]

In [18]:
rna.write('Multi-omics_intergration/data/adata_RNA_final_input.h5ad',compression="gzip")

/data/gaojie/miniforge3/envs/GLUE/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/data/gaojie/miniforge3/envs/GLUE/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/data/gaojie/miniforge3/envs/GLUE/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/data/gaojie/miniforge3/envs/GLUE/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/data/gaojie/miniforge3/envs/GLUE/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.var` of view, ini

### preparing ATAC

In [19]:
ATAC_raw_matrix = mmread('Multi-omics_intergration/data/ATAC_peak_raw_counts.mtx')

In [20]:
atac = sc.AnnData(X=ATAC_raw_matrix.T.tocsr())

In [21]:
atac

AnnData object with n_obs × n_vars = 20981 × 388257

In [22]:
atac_var = pd.read_csv('Multi-omics_intergration/data/ATAC_peak_meta.csv',index_col=0)
atac_obs = pd.read_csv('Multi-omics_intergration/data/ATAC_cell_meta.csv',index_col=0)

In [23]:
atac_var.head()

,seqnames,start,end,width,strand,idx
chr1:817083-817583,chr1,817083,817583,501,*,1
chr1:827314-827814,chr1,827314,827814,501,*,2
chr1:858598-859098,chr1,858598,859098,501,*,3
chr1:869685-870185,chr1,869685,870185,501,*,4
chr1:899841-900341,chr1,899841,900341,501,*,5


In [24]:
atac_obs.head()

,Sample,TSSEnrichment,ReadsInTSS,ReadsInPromoter,ReadsInBlacklist,PromoterRatio,PassQC,NucleosomeRatio,nMultiFrags,nMonoFrags,nFrags,nDiFrags,BlacklistRatio,doublet_pval,doublet_qval,doublet,Clusters,ReadsInPeaks,FRIP
Adjacent#ACAAGCTGTATTCTCT-1,Adjacent,4.145,7222,10386,1129,0.056009,1,0.411647,6740,65680,92717,20297,0.006088,9.999836e-01,9.999989e-01,False,C3,30731,0.165766
Adjacent#TTGAGTGAGTTACACC-1,Adjacent,9.906,26654,31016,675,0.175476,1,0.512839,5910,58418,88377,24049,0.003819,2.589578e-48,5.876057e-46,True,C3,83658,0.473313
Adjacent#GAGACTTCATTCACCC-1,Adjacent,13.267,34418,35675,794,0.206977,1,0.498461,4981,57513,86181,23687,0.004607,NaN,NaN,NaN,C6,92896,0.538965
Adjacent#CCGTGAGTCAAAGTAG-1,Adjacent,10.963,26812,30380,937,0.177686,1,0.597727,5928,53506,85488,26054,0.005480,9.281757e-60,3.768882e-57,True,C3,76780,0.449085
Adjacent#CCCTCTCTCTCTGCAC-1,Adjacent,9.936,25289,28135,676,0.165457,1,0.638726,6172,51883,85022,26967,0.003975,6.358629e-21,3.104862e-19,True,C3,71356,0.419638


In [25]:
atac.var = atac_var
atac.obs = atac_obs
atac.var.columns = ['chrom','chromStart','chromEnd','width','strand','idx']
atac.obs['doublet'] = atac.obs['doublet'].astype(str)

In [26]:
lsi = pd.read_csv('Multi-omics_intergration/data/ATAC_Harmony_LSI.csv',index_col=0) ###input the exact LSI embedding from ArchR
atac.obsm['X_lsi_harmony'] = lsi.loc[atac.obs_names,:].values

In [27]:
sc.pp.neighbors(atac, use_rep="X_lsi_harmony")
sc.tl.umap(atac)

In [28]:
atac.write('Multi-omics_intergration/data/adata_ATAC_final_input.h5ad',compression="gzip")